In [1]:
TLC_URL = 'https://d37ci6vzurychx.cloudfront.net/trip-data/'
EVENT_URL = 'https://data.cityofnewyork.us/api/views/6v4b-5gp4/rows.csv?accessType=DOWNLOAD&bom=true&format=true&delimiter=%3B'

LANDING_DATA = '../data/landing/'
ALL_SOURCES = ('yellow', 'green', 'fhvhv')


In [2]:
import os

# from the current `proj1` directory
output_relative_dir = '../data/'

# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
# now, for each type of data set we will need, we will create the paths
for target_dir in ('landing', 'raw', 'curated', 'analysis'): 
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

output_relative_dir = '../data/landing/'
for target_dir in ALL_SOURCES: 
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

In [3]:
from urllib.request import urlretrieve

def retrieve_data(source: str, year: str, month: str) -> None:
    # generate url
    get_url = f'{TLC_URL+source+"_tripdata_"}{year}-{month}.parquet'
    # generate output location and filename
    get_path = f'{LANDING_DATA+source+"/"}/{year}-{month}.parquet'
    # download
    urlretrieve(get_url, get_path)


In [4]:
import itertools

# take all data from May 2020 - May 2023
years = ['2020', '2021', '2022', '2023']
start_year = '2020'; start_month = 5
end_year = '2023'; end_month = 5
months = range(1, 13)

all_months = itertools.product(years, months)
for year_month in all_months:
    year, month = year_month
    if not((year == start_year and month < start_month) | (year == end_year and month > end_month)):
        # 0-fill i.e 1 -> 01, 2 -> 02, etc
        month = str(month).zfill(2) 
        print(f"Begin year {year} month {month}")
        for source in ALL_SOURCES:
            retrieve_data(source, year, month)        
        print(f"Completed year {year} month {month}")

Begin year 2020 month 05
Completed year 2020 month 05
Begin year 2020 month 06
Completed year 2020 month 06
Begin year 2020 month 07
Completed year 2020 month 07
Begin year 2020 month 08
Completed year 2020 month 08
Begin year 2020 month 09
Completed year 2020 month 09
Begin year 2020 month 10
Completed year 2020 month 10
Begin year 2020 month 11
Completed year 2020 month 11
Begin year 2020 month 12
Completed year 2020 month 12
Begin year 2021 month 01
Completed year 2021 month 01
Begin year 2021 month 02
Completed year 2021 month 02
Begin year 2021 month 03
Completed year 2021 month 03
Begin year 2021 month 04
Completed year 2021 month 04
Begin year 2021 month 05
Completed year 2021 month 05
Begin year 2021 month 06
Completed year 2021 month 06
Begin year 2021 month 07
Completed year 2021 month 07
Begin year 2021 month 08
Completed year 2021 month 08
Begin year 2021 month 09
Completed year 2021 month 09
Begin year 2021 month 10
Completed year 2021 month 10
Begin year 2021 month 11
Com

In [3]:
from urllib.request import urlretrieve

# generate output location and filename
get_path = f'{LANDING_DATA}Parks_Special_Events.csv'
# download
urlretrieve(EVENT_URL, get_path)

('../data/landing/Parks_Special_Events.csv',
 <http.client.HTTPMessage at 0x7fcf5c50a170>)

In [5]:
import pandas as pd

event_data = pd.read_csv(f'{LANDING_DATA}Parks_Special_Events.csv', sep=';')

In [6]:
event_data.head()

,Unit,Group Name/Partner,Date and Time,Borough,LocationType,Location,Event Name,Event Type,Category,Classification,Attendance,Audience
0,Recreation,NaN,04/12/2018 06:30:00 PM,Manhattan,Recreation Center,Alfred E. Smith Recreation Center,Battle of the Paddles Special Event,Tournament/Competition,Sport,NaN,60,Adults;#Seniors;#Young Adult
1,Recreation,NaN,01/20/2018 12:00:00 AM,Manhattan,Recreation Center,Alfred E. Smith Recreation Center,Event New Year Ultimate Fitness Frenzy,Local Event,Fitness,NaN,50,Teens;#Young Adult;#Adults;#Seniors
2,Recreation,NaN,06/11/2018 10:00:00 AM,Bronx,Park,Poe Park,Youth Arts Festival,Community Based Event,Mobile Unit,Play,150,Children;#General Public
3,Recreation,Musuem of Chinese in America,06/07/2019 07:00:00 PM,Manhattan,Park,Columbus Park,Movie: Kung Fu Panda,Community Based Event,Mobile Unit,M.U.T.S,325,Children;#Young Adult;#General Public
4,Recreation,NaN,06/08/2019 11:00:00 AM,Manhattan,Recreation Center,Pelham Fritz Recreation Center,"Live, Laugh, Love Fitness Expo",Agency Produced Event,Fitness,Fitness,166,General Public;#Adults;#Young Adult;#Seniors


In [13]:
import os
from datetime import date, datetime, timedelta

import requests


def fetch_weather_data(station_id):
    """Download data we are interested in!"""
    localfn = f"{LANDING_DATA}{station_id}.csv"
    if os.path.isfile(localfn):
        print(f"- Cowardly refusing to over-write existing file: {localfn}")
        return
    print(f"+ Downloading for {station_id}")
    enddt = date.today() + timedelta(days=2)
    uri = (
        "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"
        f"station={station_id}&data=tmpc&data=sped&data=p01m&year1=2021&month1=5&day1=1&"
        f"year2=2023&month2=5&day2=31&"
        "tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&"
        "direct=yes&report_type=3"
    )
    res = requests.get(uri, timeout=300)
    with open(localfn, "w", encoding="utf-8") as fh:
        fh.write(res.text)


In [14]:
fetch_weather_data('JFK')

+ Downloading for JFK


In [41]:
weather_data = pd.read_csv(f'{LANDING_DATA}JFK.csv', sep=',')

In [53]:



    print('cat')

cat


In [46]:
weather_data.loc[0,'sped'].isdecimal()

False

In [30]:
def isfloat(value): 
    try: 
        return True, float(value) 
    except ValueError: 
        return False, value 

def fill_missing(data: pd.Series) -> pd.Series:
    # find first valid entry
    for row, entry in enumerate(data):
        found_float, first_float = isfloat(entry)
        if found_float:
            break
    
    for row, entry in enumerate(data):
        if ~isfloat(entry[0]):
            if row == 0:

            elif row == data.size:

            else:
                data[row] = 


station     object
valid       object
tmpc       float64
sped        object
p01m        object
dtype: object

In [33]:
weather_data[['sped', 'p01m']].replace('T',0, inplace=True)
weather_data[['sped', 'p01m']].replace('M',0, inplace=True)


/tmp/ipykernel_92001/3165268751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data[['sped', 'p01m']].replace('T',0, inplace=True)
/tmp/ipykernel_92001/3165268751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data[['sped', 'p01m']].replace('M',0, inplace=True)


In [ ]:
weather_data[['sped', 'p01m']] = weather_data[['sped', 'p01m']].apply(pd.to_numeric)

In [27]:
import numpy as np
weather_data[~weather_data.loc[:, weather_data.columns.isin(['tmpc', 'sped', 'p01m'])].applymap(np.isreal).all(1)]

,station,valid,tmpc,sped,p01m
0,JFK,2021-05-01 00:51,12.78,31.05,T
1,JFK,2021-05-01 01:51,11.11,20.70,0.00
2,JFK,2021-05-01 02:51,10.00,28.75,0.00
3,JFK,2021-05-01 03:51,8.89,26.45,0.00
4,JFK,2021-05-01 04:51,7.78,29.90,0.00
...,...,...,...,...,...
18224,JFK,2023-05-30 19:51,17.78,9.20,0.00
18225,JFK,2023-05-30 20:51,18.28,12.65,0.00
18226,JFK,2023-05-30 21:51,16.72,12.65,0.00
18227,JFK,2023-05-30 22:51,15.61,11.50,0.00


In [22]:
weather_data[['tmpc', 'sped', 'p01m']].describe()

,tmpc
count,18229.000000
mean,13.556579
std,9.077536
min,-15.000000
25%,6.720000
50%,13.280000
75%,21.110000
max,35.000000


In [23]:
weather_data[['tmpc', 'sped', 'p01m']].describe()
weather_data[['sped']].describe()

,sped
count,18229
unique,35
top,10.35
freq,1531
